<a href="https://colab.research.google.com/github/velievarif123-sudo/intro-ai/blob/lab1/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22lab1_regression_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа № 1 Задача регрессии

## Задание
Решите пожалуйста следующие задачи.

1. Примените следующие типы регрессий для датасета представленного по
[ссылке](https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link).
- Линейная регрессия
- LASSO регрессия
- Ridge регрессия
- ElasticNet регрессия
- Polynom регрессия (для степени полинома 2)
- Polynom регрессия (для степени полинома 5)
- Polynom регрессия (для степени полинома 10)

Постройте модель для атрибутов `X = {SkinThickness, BMI}` и `Y = {Insulin}`.  
**Укажите `score` (r2_score) моделей. Найдите метрики MAE, MSE.**


## Загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 5.89MB/s]


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


##  О датасете
[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data)

Этот набор данных взят из Национального института диабета, заболеваний органов пищеварения и почек (NIDDK). Целью набора данных является диагностическое предсказание наличия у пациента диабета на основе определенных диагностических измерений, включенных в набор данных. На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года, родом из индейцев Пима.

Наборы данных состоят из нескольких переменных медицинских предикторов и одной целевой переменной `Outcome`. Переменные-предикторы включают количество беременностей, которые были у пациентки, её индекс массы тела `BMI`, уровень инсулина, возраст и так далее. Всего в датасете 9 переменных:
- `Pregnancies` - количество беременностей;
- `Glucose` - концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе;
- `BloodPressure` - диастолическое артериальное давление (мм рт. ст.);
- `SkinThickness` - толщина кожной складки трицепса (мм);
- `Insulin` - 2-часовой сывороточный инсулин (мкЕд/мл);
- `BMI` - Индекс массы тела (вес в кг/(рост в м)^2);
- `DiabetesPedigreeFunction` - функция оценки вероятности диабета на основе семейного анамнеза;
- `Age` - возраст (лет);
- `Outcome` - целевая переменная (0 или 1) 268 из 768 — 1 (больны), остальные — 0 (здоровы).



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Линейная регрессия

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y) - using 'Insulin' as target
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin'] # Using 'Insulin' as target

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Scale features (optional but good practice for many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, Y_train)

# Make predictions and evaluate the model
Y_pred_linear = linear_model.predict(X_test_scaled)
score_linear = r2_score(Y_test, Y_pred_linear)

print(f"Linear Regression R2 Score for Insulin: {score_linear}")

Linear Regression R2 Score for Insulin: 0.20047884265876637


## LASSO регрессия

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y)
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the LASSO regression model
lasso_model = Lasso(alpha=1.0) # You can adjust the alpha value
lasso_model.fit(X_train_scaled, Y_train)

# Make predictions and evaluate the model
Y_pred_lasso = lasso_model.predict(X_test_scaled)
score_lasso = r2_score(Y_test, Y_pred_lasso)

print(f"LASSO Regression R2 Score: {score_lasso}")

LASSO Regression R2 Score: 0.19925659255624573


## Ridge регрессия

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y)
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Ridge regression model
ridge_model = Ridge(alpha=1.0) # You can adjust the alpha value
ridge_model.fit(X_train_scaled, Y_train)

# Make predictions and evaluate the model
Y_pred_ridge = ridge_model.predict(X_test_scaled)
score_ridge = r2_score(Y_test, Y_pred_ridge)

print(f"Ridge Regression R2 Score: {score_ridge}")

Ridge Regression R2 Score: 0.2004267199549601


## ElasticNet регрессия

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y)
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the ElasticNet regression model
elasticnet_model = ElasticNet(alpha=1.0, l1_ratio=0.5) # You can adjust alpha and l1_ratio
elasticnet_model.fit(X_train_scaled, Y_train)

# Make predictions and evaluate the model
Y_pred_elasticnet = elasticnet_model.predict(X_test_scaled)
score_elasticnet = r2_score(Y_test, Y_pred_elasticnet)

print(f"ElasticNet Regression R2 Score: {score_elasticnet}")

ElasticNet Regression R2 Score: 0.1702521548376016


## Polynom регрессия (для степени полинома 2)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y)
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Create polynomial features
poly_features_2 = PolynomialFeatures(degree=2)
X_train_poly_2 = poly_features_2.fit_transform(X_train)
X_test_poly_2 = poly_features_2.transform(X_test)

# Scale polynomial features
scaler_poly_2 = StandardScaler()
X_train_poly_scaled_2 = scaler_poly_2.fit_transform(X_train_poly_2)
X_test_poly_scaled_2 = scaler_poly_2.transform(X_test_poly_2)

# Create and train the Linear Regression model on polynomial features
poly_model_2 = LinearRegression()
poly_model_2.fit(X_train_poly_scaled_2, Y_train)

# Make predictions and evaluate the model
Y_pred_poly_2 = poly_model_2.predict(X_test_poly_scaled_2)
score_poly_2 = r2_score(Y_test, Y_pred_poly_2)

print(f"Polynomial Regression (Degree 2) R2 Score: {score_poly_2}")

Polynomial Regression (Degree 2) R2 Score: 0.21579896816428001


## Polynom регрессия (для степени полинома 5)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y)
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Create polynomial features
poly_features_5 = PolynomialFeatures(degree=5)
X_train_poly_5 = poly_features_5.fit_transform(X_train)
X_test_poly_5 = poly_features_5.transform(X_test)

# Scale polynomial features
scaler_poly_5 = StandardScaler()
X_train_poly_scaled_5 = scaler_poly_5.fit_transform(X_train_poly_5)
X_test_poly_scaled_5 = scaler_poly_5.transform(X_test_poly_5)

# Create and train the Linear Regression model on polynomial features
poly_model_5 = LinearRegression()
poly_model_5.fit(X_train_poly_scaled_5, Y_train)

# Make predictions and evaluate the model
Y_pred_poly_5 = poly_model_5.predict(X_test_poly_scaled_5)
score_poly_5 = r2_score(Y_test, Y_pred_poly_5)

print(f"Polynomial Regression (Degree 5) R2 Score: {score_poly_5}")

Polynomial Regression (Degree 5) R2 Score: -0.0005003389887692222


## Polynom регрессия (для степени полинома 10)

In [ ]:

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Select features (X) and target (Y)
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)

# Create polynomial features
poly_features_10 = PolynomialFeatures(degree=10)
X_train_poly_10 = poly_features_10.fit_transform(X_train)
X_test_poly_10 = poly_features_10.transform(X_test)

# Scale polynomial features
scaler_poly_10 = StandardScaler()
X_train_poly_scaled_10 = scaler_poly_10.fit_transform(X_train_poly_10)
X_test_poly_scaled_10 = scaler_poly_10.transform(X_test_poly_10)

# Create and train the Linear Regression model on polynomial features
poly_model_10 = LinearRegression()
poly_model_10.fit(X_train_poly_scaled_10, Y_train)

# Make predictions and evaluate the model
Y_pred_poly_10 = poly_model_10.predict(X_test_poly_scaled_10)
score_poly_10 = r2_score(Y_test, Y_pred_poly_10)

print(f"Polynomial Regression (Degree 10) R2 Score: {score_poly_10}")

Polynomial Regression (Degree 10) R2 Score: -17.373345207103128


## Вывод

Если сравнивать модели по коэффициенту R2, то видно, что лучшее качество показала полиномиальная регрессия второй степени. Она немного обошла обычную линейную регрессию и модели с регуляризацией (Ridge, LASSO и ElasticNet). Скорее всего, это связано с тем, что зависимость между признаками SkinThickness, BMI и уровнем инсулина не совсем линейная, и полиномиальная модель лучше её учитывает.

Модели Ridge и LASSO дали почти такие же результаты, как и линейная регрессия, подчеркнув, что в данной задаче регуляризация не даёт заметного преимущества. ElasticNet, наоборот, показал худший результат среди этих моделей.

Полиномиальные регрессии высокой степени (5 и 10) показали очень плохие значения R2, вплоть до отрицательных. Это говорит о том, что такие модели сильно переобучаются и плохо работают на новых данных, поэтому использовать их в данном случае нецелесообразно.

В итоге можно сказать, что для этого набора данных наиболее удачным вариантом оказалась полиномиальная регрессия второй степени. При этом в целом низкие значения R2 у всех моделей показывают, что признаки SkinThickness и BMI сами по себе не очень хорошо объясняют уровень инсулина, и для более точного предсказания, скорее всего, нужны дополнительные признаки или более сложные модели.